In [ ]:
import os
import ee
import geemap

ee.Authenticate()
ee.Initialize()

In [ ]:
Map = geemap.Map()
Map

In [ ]:
region_shp=r'[shp]'

In [ ]:
region=geemap.shp_to_ee(region_shp)

In [ ]:
Map.addLayer(region,{},'region',True)

In [ ]:
features = geemap.fishnet(region, rows=2, cols=2,delta=0.5)
Map.addLayer(features, {}, 'Grids',True)

In [ ]:
print(features.size().getInfo())

In [ ]:
def maskS2clouds(image):
    return image.divide(1000)

def fmask(image):
    qaMask = image.select('QA60').bitwiseAnd(int('11111', 10)).eq(0)
    qaMask2 = image.select('QA60').bitwiseAnd(int('11111', 11)).eq(0)


    return image.updateMask(qaMask).divide(1000)

In [ ]:

wkt='''PROJCS["WGS 84 / Pseudo-Mercator",
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4326"]],
    PROJECTION["Mercator_1SP"],
    PARAMETER["central_meridian",0],
    PARAMETER["scale_factor",1],
    PARAMETER["false_easting",0],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH],
    EXTENSION["PROJ4","+proj=merc +a=6378137 +b=6378137 +lat_ts=0 +lon_0=0 +x_0=0 +y_0=0 +k=1 +units=m +nadgrids=@null +wktext +no_defs"],
    AUTHORITY["EPSG","3857"]]'''

proj=ee.Projection(wkt)

In [ ]:
collection=ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate('2017-08-09', '2017-10-01').filterBounds(region).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))
print(collection.size().getInfo())

In [ ]:
s2=collection.min().select(['B1','B2','B3','B4','B5','B6','B7','B8','B9','B10','B11','B12']).clip(region).reproject(proj,None,10)
print(s2.getInfo())

In [ ]:
dataset = ee.Image('NASA/NASADEM_HGT/001')
elevation = dataset.select('elevation').clip(region).reproject(proj,None,10)
slope = ee.Terrain.slope(dataset.select('elevation')).clip(region).reproject(proj,None,10)



In [ ]:
out_dir_ele = os.path.expanduser(r'D:\Elevation\\')
out_dir_slo = os.path.expanduser(r'D:\Slope\\')


In [ ]:
geemap.download_ee_image_tiles(
    elevation, features, out_dir_ele, prefix="Elevation_", crs=proj, scale=10,overwrite=False
)

In [ ]:
geemap.download_ee_image_tiles(
    slope, features, out_dir_slo, prefix="Slope_", crs=proj, scale=10,overwrite=False
)

In [ ]:
out_dir=os.path.expanduser(r'D:\S2')

In [ ]:
geemap.download_ee_image_tiles(
    s2, features, out_dir, prefix="S2_", crs=proj,scale=10,overwrite=False
)